In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import json
from datetime import datetime

In [2]:
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("PIL (Pillow) available")
print("All libraries imported successfully!")

OpenCV version: 4.11.0
NumPy version: 1.26.4
PIL (Pillow) available
All libraries imported successfully!


In [3]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Created 'data' directory for storing face images")

if not os.path.exists("screenshots"):
    os.makedirs("screenshots")
    print("Created 'screenshots' directory")

Created 'data' directory for storing face images
Created 'screenshots' directory


In [4]:
def create_users_config():
    users_config = "users.json"

    default_users = {
        "1": "Kalhara",
        "2": "User2",
        "3": "User3"
    }

    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
        print("Loaded existing users configuration:")
    else:
        users = default_users
        with open(users_config, 'w') as f:
            json.dump(users, f, indent=2)
        print("Created new users configuration:")

    for user_id, name in users.items():
        print(f"    ID: {user_id} - Name: {name}")
    
    return users

In [5]:
def add_user(user_id, name):
    users_config = "users.json"
    if os.path.exists(users_config):
        with open(users_config, 'r') as f:
            users = json.load(f)
    else:
        users = {}
    users[str(user_id)] = name
    with open(users_config, 'w') as f:
        json.dump(users, f, indent=2)
    
    print(f" Added user: {name} (ID: {user_id})")
    return users
        

In [6]:
users = create_users_config()

Created new users configuration:
    ID: 1 - Name: Kalhara
    ID: 2 - Name: User2
    ID: 3 - Name: User3


In [7]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [ ]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_classifier.detectMultiScale(
        gray, 
        scaleFactor=1.3,    
        minNeighbors=5,     
        minSize=(30, 30)   
    )

    
    